In [12]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the base directory containing the species folders
BASE_DIR = r"C:\Users\BHEEMA RAJU\Desktop\AnimalSpecieDetection\AnimalSpecies"

# Directories to create for train and validation datasets
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "validation")

# Create train and validation directories if they don't exist
os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)

# Loop through species folders in BASE_DIR
for species in os.listdir(BASE_DIR):
    species_path = os.path.join(BASE_DIR, species)

    if os.path.isdir(species_path):  # Ensure it's a folder (skip files)
        print(f"Processing species: {species}")

        # Create subdirectories for each species in train and validation folders
        os.makedirs(os.path.join(TRAIN_DIR, species), exist_ok=True)
        os.makedirs(os.path.join(VAL_DIR, species), exist_ok=True)

        # List all images in the species folder
        images = os.listdir(species_path)
        images = [img for img in images if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
        print(f"Found {len(images)} images for species: {species}")

        if len(images) > 0:
            # Split the images into train and validation sets (80-20 split)
            train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

            # Move images to respective directories
            for img in train_images:
                shutil.move(os.path.join(species_path, img), os.path.join(TRAIN_DIR, species, img))
            for img in val_images:
                shutil.move(os.path.join(species_path, img), os.path.join(VAL_DIR, species, img))
        else:
            print(f"No valid images found for species: {species}")


Processing species: butterfly
Found 2112 images for species: butterfly
Processing species: cat
Found 1668 images for species: cat
Processing species: cow
Found 1866 images for species: cow
Processing species: dog
Found 4863 images for species: dog
Processing species: elephant
Found 1446 images for species: elephant
Processing species: hen
Found 3098 images for species: hen
Processing species: hourse
Found 2623 images for species: hourse
Processing species: scoiattolo
Found 1862 images for species: scoiattolo
Processing species: sheep
Found 1820 images for species: sheep
Processing species: spider
Found 4821 images for species: spider
Processing species: train
Found 0 images for species: train
No valid images found for species: train
Processing species: validation
Found 0 images for species: validation
No valid images found for species: validation


In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define image dimensions
IMG_HEIGHT = 224
IMG_WIDTH = 224

# Define data directories
TRAIN_DIR = r"C:\Users\BHEEMA RAJU\Desktop\AnimalSpecieDetection\AnimalSpecies\train"
VAL_DIR = r"C:\Users\BHEEMA RAJU\Desktop\AnimalSpecieDetection\AnimalSpecies\validation"

# Data augmentation and preprocessing for training and validation datasets
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    rotation_range=30,  # Random rotation
    width_shift_range=0.2,  # Horizontal shift
    height_shift_range=0.2,  # Vertical shift
    shear_range=0.2,  # Shearing
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Horizontal flip
    fill_mode="nearest",  # Filling for empty pixels after rotation/shift
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Only normalization for validation

# Generate datasets
train_data = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode="categorical",
)

val_data = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode="categorical",
)


Found 20938 images belonging to 10 classes.
Found 5241 images belonging to 10 classes.


In [24]:
from tensorflow.keras.layers import Layer, Input, GlobalAveragePooling2D, Dense, Reshape
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.applications import ResNet50

# Define the attention mechanism as a custom layer
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="random_normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,),
                                 initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        alpha = K.softmax(e, axis=1)
        output = x * alpha
        return K.sum(output, axis=1)

# Define image dimensions
IMG_HEIGHT = 224
IMG_WIDTH = 224

# Load pre-trained ResNet-50 model without the top layer
base_model = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)))

# Freeze the base model layers to retain pre-trained weights
base_model.trainable = False

# Add custom layers for our classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling

# Add the attention mechanism after pooling
x = AttentionLayer()(x)  # Attention layer

# Reshape the output if necessary (ensure it's 2D for the Dense layer)
x = Reshape((-1,))(x)  # Flatten the output to 1D for Dense layer

# Fully connected layer with 256 neurons
x = Dense(256, activation="relu")(x)

# Output layer (10 classes, change as needed)
output_layer = Dense(10, activation="softmax")(x)

# Build the complete model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Model summary
model.summary()

# Save the model to a file
model_save_path = 'species_detection_model_with_attention.h5'  # Define the path where you want to save the model
model.save(model_save_path)

print(f"Model saved at: {model_save_path}")


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_6[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

                                                                                                  
 conv2_block3_1_conv (Conv2  (None, 56, 56, 64)           16448     ['conv2_block2_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv2_block3_1_bn (BatchNo  (None, 56, 56, 64)           256       ['conv2_block3_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv2_block3_1_relu (Activ  (None, 56, 56, 64)           0         ['conv2_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv2_blo

 ation)                                                                                           
                                                                                                  
 conv3_block2_3_conv (Conv2  (None, 28, 28, 512)          66048     ['conv3_block2_2_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block2_3_bn (BatchNo  (None, 28, 28, 512)          2048      ['conv3_block2_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv3_block2_add (Add)      (None, 28, 28, 512)          0         ['conv3_block1_out[0][0]',    
                                                                     'conv3_block2_3_bn[0][0]']   
          

 conv4_block1_2_conv (Conv2  (None, 14, 14, 256)          590080    ['conv4_block1_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv4_block1_2_bn (BatchNo  (None, 14, 14, 256)          1024      ['conv4_block1_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block1_2_relu (Activ  (None, 14, 14, 256)          0         ['conv4_block1_2_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv4_block1_0_conv (Conv2  (None, 14, 14, 1024)         525312    ['conv3_block4_out[0][0]']    
 D)       

                                                                                                  
 conv4_block3_out (Activati  (None, 14, 14, 1024)         0         ['conv4_block3_add[0][0]']    
 on)                                                                                              
                                                                                                  
 conv4_block4_1_conv (Conv2  (None, 14, 14, 256)          262400    ['conv4_block3_out[0][0]']    
 D)                                                                                               
                                                                                                  
 conv4_block4_1_bn (BatchNo  (None, 14, 14, 256)          1024      ['conv4_block4_1_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_blo

 D)                                                                                               
                                                                                                  
 conv4_block6_3_bn (BatchNo  (None, 14, 14, 1024)         4096      ['conv4_block6_3_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv4_block6_add (Add)      (None, 14, 14, 1024)         0         ['conv4_block5_out[0][0]',    
                                                                     'conv4_block6_3_bn[0][0]']   
                                                                                                  
 conv4_block6_out (Activati  (None, 14, 14, 1024)         0         ['conv4_block6_add[0][0]']    
 on)                                                                                              
          

 conv5_block3_1_relu (Activ  (None, 7, 7, 512)            0         ['conv5_block3_1_bn[0][0]']   
 ation)                                                                                           
                                                                                                  
 conv5_block3_2_conv (Conv2  (None, 7, 7, 512)            2359808   ['conv5_block3_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv5_block3_2_bn (BatchNo  (None, 7, 7, 512)            2048      ['conv5_block3_2_conv[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv5_block3_2_relu (Activ  (None, 7, 7, 512)            0         ['conv5_block3_2_bn[0][0]']   
 ation)   

In [ ]:
# Train only the top layers first
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,
    steps_per_epoch=train_data.samples // train_data.batch_size,
    validation_steps=val_data.samples // val_data.batch_size,
)

# Unfreeze base_model and fine-tune
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Fine-tuning
history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    steps_per_epoch=train_data.samples // train_data.batch_size,
    validation_steps=val_data.samples // val_data.batch_size,
)


Epoch 1/5
654/654 [==============================] - 1392s 2s/step - loss: 2.4811 - accuracy: 0.1550 - val_loss: 2.4935 - val_accuracy: 0.1181
Epoch 2/5
654/654 [==============================] - 1453s 2s/step - loss: 2.4975 - accuracy: 0.1485 - val_loss: 2.5462 - val_accuracy: 0.0807
Epoch 3/5
654/654 [==============================] - 1344s 2s/step - loss: 2.4386 - accuracy: 0.1592 - val_loss: 2.3218 - val_accuracy: 0.1846
Epoch 4/5
654/654 [==============================] - 1275s 2s/step - loss: 2.4446 - accuracy: 0.1540 - val_loss: 2.5700 - val_accuracy: 0.1183
Epoch 5/5
654/654 [==============================] - 3950s 6s/step - loss: 2.4305 - accuracy: 0.1570 - val_loss: 2.4713 - val_accuracy: 0.1854
Epoch 1/10

265/654 [===========>..................] - ETA: 31:24 - loss: 2.2612 - accuracy: 0.1735